## **Settings**

In [2]:
import os
from sys import platform
from utils.global_config import get_global_config
from utils.utils import make_dataset, get_raw_data, get_train_test_data, train_test_model, save_results

In [3]:
_config = get_global_config()
BASE_PATH = '\\'.join(os.getcwd().split('\\')[:-1]) + '\\' if platform == 'win32' else '/'.join(os.getcwd().split('/')[:-1]) + '/'
DATA_PATH = os.path.join(BASE_PATH, _config.get('DATA_PATH'))
REPORT_FILE = os.path.join(BASE_PATH, _config.get('RESULT_FILE_DEFAULT'))
if platform == 'win32':
	DATA_PATH = DATA_PATH.replace('/', '\\')
	REPORT_FILE = REPORT_FILE.replace('/', '\\')
REMOTE_DATA = _config.get('REMOTE_DATA')
VIEW = _config.get('VIEW')
EXCHANGE = _config.get('EXCHANGE')
CRYPTO = _config.get('CRYPTO')
FEATURES_EXCLUDED = _config.get('FEATURES_EXCLUDED')

In [3]:
# Uncomment this to import the data

# make_dataset(REMOTE_DATA, DATA_PATH)

In [4]:
# Uncomment this to check a sample data

# candle = get_raw_data(DATA_PATH, VIEW[0], EXCHANGE[0], CRYPTO[0])
# snapshot = get_raw_data(DATA_PATH, VIEW[1], EXCHANGE[0], CRYPTO[0])
# candle.sample(n=5)
# snapshot.sample(n=5)

In [24]:
# Printing of classes distributions for each dataset

print(''.join(['> ' for i in range(40)]))
print(f'\n{"VIEW":>10}{"EXCHANGE":>10}{"CRYPTO":>10}{"DOWN":>18}{"NEUTRAL":>15}{"UP":>15}\n')
print(''.join(['> ' for i in range(40)]))
for v in VIEW:
	for e in EXCHANGE:
		for c in CRYPTO:
			data = get_raw_data(DATA_PATH, v, e, c)
			support = data['labels'].value_counts()
			d, n, u = support['down'], support['neutral'], support['up']
			_d, _n, _u = round(d * 100 / (d+n+u), 1), round(n * 100 / (d+n+u), 1), round(u * 100 / (d+n+u), 1)
			print(f'{v.upper():>10}{e.upper():>10}{c.upper():>10}{str(d)+" ("+str(_d)+"%)":>18}{str(n)+" ("+str(_n)+"%)":>15}{str(u)+" ("+str(_u)+"%)":>15}')

> > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > 

      VIEW  EXCHANGE    CRYPTO              DOWN        NEUTRAL             UP

> > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > 
   CANDLES   BINANCE      AVAX     68408 (14.2%) 340097 (70.7%)  72395 (15.1%)
   CANDLES   BINANCE       BNB     69482 (14.5%) 343118 (71.8%)  65420 (13.7%)
   CANDLES   BINANCE      DOGE     71291 (14.9%) 336601 (70.1%)  72006 (15.0%)
   CANDLES   BINANCE       DOT     64452 (13.4%) 349548 (72.7%)  66900 (13.9%)
   CANDLES   BINANCE       NEO     50782 (10.6%) 381230 (79.8%)   45998 (9.6%)
   CANDLES   BINANCE       SOL     73210 (15.2%) 337995 (70.3%)  69695 (14.5%)
   CANDLES     HOUBI      AVAX     44438 (11.3%) 301665 (76.7%)  46956 (11.9%)
   CANDLES     HOUBI       BNB     47649 (12.1%) 299989 (76.3%)  45419 (11.6%)
   CANDLES     HOUBI      DOGE     50949 (13.0%) 290424 (73.9%)  51687 (13.1%)
   CANDLES     HOUBI       DOT     47303 (12.0

## **Group 1**

Executing experiments on single views (i.e. **candle** or **snapshot**).

In [4]:
counter = 1
for v in VIEW:
	for e in EXCHANGE:
		for c in CRYPTO:
			print(f'{"Executing ("+str(counter)+"/"+str(len(VIEW)*len(EXCHANGE)*len(CRYPTO))+")":<20}{v.upper():>10}{e.upper():>10}{c.upper():>10}')
			data = get_raw_data(DATA_PATH, v, e, c)
			X_train, y_train, X_test, y_test = get_train_test_data(data, FEATURES_EXCLUDED[v], verbose=False)
			del data
			results = train_test_model(X_train, y_train, X_test, y_test)
			save_results(REPORT_FILE, results, v+'-'+e, c)
			counter += 1

Executing (1/36)       CANDLES   BINANCE      AVAX
Executing (1/36)       CANDLES   BINANCE       BNB
Executing (1/36)       CANDLES   BINANCE      DOGE
Executing (1/36)       CANDLES   BINANCE       DOT
Executing (1/36)       CANDLES   BINANCE       NEO
Executing (1/36)       CANDLES   BINANCE       SOL
Executing (1/36)       CANDLES     HOUBI      AVAX
Executing (1/36)       CANDLES     HOUBI       BNB
Executing (1/36)       CANDLES     HOUBI      DOGE
Executing (1/36)       CANDLES     HOUBI       DOT
Executing (1/36)       CANDLES     HOUBI       NEO
Executing (1/36)       CANDLES     HOUBI       SOL
Executing (1/36)       CANDLES       OKX      AVAX
Executing (1/36)       CANDLES       OKX       BNB
Executing (1/36)       CANDLES       OKX      DOGE
Executing (1/36)       CANDLES       OKX       DOT
Executing (1/36)       CANDLES       OKX       NEO
Executing (1/36)       CANDLES       OKX       SOL
Executing (1/36)      SNAPSHOT   BINANCE      AVAX
Executing (1/36)      SNAPSHOT 

## **Group 2**

Executing experiments on double views of the same type (i.e. **candle-candle** or **snapshot-snapshot**).

In [4]:
view_id = {'1': 'BINANCE-HOUBI', '2': 'BINANCE-OKX', '3': 'HOUBI-OKX'}
counter = 1
for v in VIEW:
	columns_to_drop = FEATURES_EXCLUDED[v] + [i+'_1' for i in FEATURES_EXCLUDED[v][1:]]
	for c in CRYPTO:
		for mv in view_id.values():
			view_name = v + '-' + mv.split('-')[0] + '--' + v + '-' + mv.split('-')[1]
			print(f'{"Executing ("+str(counter)+"/"+str(len(VIEW)*len(view_id)*len(CRYPTO))+")":<20}{view_name:<35}{c.upper():<15}')
			left = get_raw_data(DATA_PATH, v, mv.split('-')[0], c)
			right = get_raw_data(DATA_PATH, v, mv.split('-')[1], c)
			merge_side = 'left' if len(left) > len(right) else 'right'
			data = left.merge(right, on='origin_time', how=merge_side, suffixes=('', '_1'))
			X_train, y_train, X_test, y_test = get_train_test_data(data, columns_to_drop)
			del left, right, data
			results = train_test_model(X_train, y_train, X_test, y_test)
			save_results(REPORT_FILE, results, view_name, c)
			counter += 1

Executing (1/36)    candles-BINANCE--candles-HOUBI     AVAX           
Executing (2/36)    candles-BINANCE--candles-OKX       AVAX           
Executing (3/36)    candles-HOUBI--candles-OKX         AVAX           
Executing (4/36)    candles-BINANCE--candles-HOUBI     BNB            
Executing (5/36)    candles-BINANCE--candles-OKX       BNB            
Executing (6/36)    candles-HOUBI--candles-OKX         BNB            
Executing (7/36)    candles-BINANCE--candles-HOUBI     DOGE           
Executing (8/36)    candles-BINANCE--candles-OKX       DOGE           
Executing (9/36)    candles-HOUBI--candles-OKX         DOGE           
Executing (10/36)   candles-BINANCE--candles-HOUBI     DOT            
Executing (11/36)   candles-BINANCE--candles-OKX       DOT            
Executing (12/36)   candles-HOUBI--candles-OKX         DOT            
Executing (13/36)   candles-BINANCE--candles-HOUBI     NEO            
Executing (14/36)   candles-BINANCE--candles-OKX       NEO            
Execut

## **Group 3**

Executing experiments on double views of different type (i.e. **snapshop-candle**).

In [10]:
counter = 1
for e in EXCHANGE:
	for c in CRYPTO:
		columns_to_drop = FEATURES_EXCLUDED[VIEW[1]] + ['start', 'stop', 'exchange_1', 'symbol_1', 'labels_1']
		view_name = VIEW[1] + '-' + e + '--' + VIEW[0] + '-' + e
		print(f'{"Executing ("+str(counter)+"/"+str(len(EXCHANGE)*len(CRYPTO))+")":<20}{view_name:<35}{c.upper():<15}')
		snapshot = get_raw_data(DATA_PATH, VIEW[1], e, c)
		candle = get_raw_data(DATA_PATH, VIEW[0], e, c)
		data = snapshot.merge(candle, on='origin_time', how='left', suffixes=('', '_1'))
		X_train, y_train, X_test, y_test = get_train_test_data(data, columns_to_drop)
		del snapshot, candle, data
		results = train_test_model(X_train, y_train, X_test, y_test)
		save_results(REPORT_FILE, results, view_name, c)
		counter += 1

Executing (1/18)    snapshot-BINANCE--candles-BINANCE  AVAX           
Executing (2/18)    snapshot-BINANCE--candles-BINANCE  BNB            
Executing (3/18)    snapshot-BINANCE--candles-BINANCE  DOGE           
Executing (4/18)    snapshot-BINANCE--candles-BINANCE  DOT            
Executing (5/18)    snapshot-BINANCE--candles-BINANCE  NEO            
Executing (6/18)    snapshot-BINANCE--candles-BINANCE  SOL            
Executing (7/18)    snapshot-HOUBI--candles-HOUBI      AVAX           
Executing (8/18)    snapshot-HOUBI--candles-HOUBI      BNB            
Executing (9/18)    snapshot-HOUBI--candles-HOUBI      DOGE           
Executing (10/18)   snapshot-HOUBI--candles-HOUBI      DOT            
Executing (11/18)   snapshot-HOUBI--candles-HOUBI      NEO            
Executing (12/18)   snapshot-HOUBI--candles-HOUBI      SOL            
Executing (13/18)   snapshot-OKX--candles-OKX          AVAX           
Executing (14/18)   snapshot-OKX--candles-OKX          BNB            
Execut

## **Group 4**

Executing experiments on triple views of the same type (i.e. **snapshop-snapshot-snapshot** or **candle-candle-candle**).

In [4]:
counter = 1
for v in [VIEW[0]]:  # WARNING: OUT-OF-MEMORY EXPERIMENT FROM VIEW[1]
	for c in CRYPTO:
		columns_to_drop = FEATURES_EXCLUDED[v] + [i+'_1' for i in FEATURES_EXCLUDED[v][1:]] + [i+'_2' for i in FEATURES_EXCLUDED[v][1:]]
		view_name = v + '-' + EXCHANGE[0] + '--' + v + '-' + EXCHANGE[1] + '--' + v + '-' + EXCHANGE[2]
		print(f'{"Executing ("+str(counter)+"/"+str(len(VIEW)*len(CRYPTO))+")":<20}{view_name:<50}{c.upper():<15}')
		binance = get_raw_data(DATA_PATH, v, EXCHANGE[0], c)
		houbi = get_raw_data(DATA_PATH, v, EXCHANGE[1], c)
		okx = get_raw_data(DATA_PATH, v, EXCHANGE[2], c)
		data = binance.merge(houbi, on='origin_time', how='left', suffixes=('', '_1'))
		data = data.merge(okx, on='origin_time', how='left', suffixes=('', '_2'))
		X_train, y_train, X_test, y_test = get_train_test_data(data, columns_to_drop)
		del binance, houbi, okx, data
		results = train_test_model(X_train, y_train, X_test, y_test)
		save_results(REPORT_FILE, results, view_name, c)
		counter += 1

Executing (1/12)    candles-BINANCE--candles-HOUBI--candles-OKX       AVAX           
Executing (2/12)    candles-BINANCE--candles-HOUBI--candles-OKX       BNB            
Executing (3/12)    candles-BINANCE--candles-HOUBI--candles-OKX       DOGE           
Executing (4/12)    candles-BINANCE--candles-HOUBI--candles-OKX       DOT            
Executing (5/12)    candles-BINANCE--candles-HOUBI--candles-OKX       NEO            
Executing (6/12)    candles-BINANCE--candles-HOUBI--candles-OKX       SOL            


## **Group 5**

Executing experiments on quadruple views of different couples of the same type (i.e. **snapshop-snapshot-candle-candle** ).

In [20]:
# WARNING: OUT-OF-MEMORY EXPERIMENT

view_id = {'1': 'BINANCE-HOUBI', '2': 'BINANCE-OKX', '3': 'HOUBI-OKX'}
counter = 1
columns_to_drop = FEATURES_EXCLUDED[VIEW[1]] + [i+'_1' for i in FEATURES_EXCLUDED[VIEW[1]][1:]] + FEATURES_EXCLUDED[VIEW[0]] + [i+'_1' for i in FEATURES_EXCLUDED[VIEW[0]][1:]]
columns_to_drop = list(set(columns_to_drop))
for c in CRYPTO:
	for mv in view_id.values():
		view_name = VIEW[1] + '-' + mv.split('-')[0] + '--' + VIEW[1] + '-' + mv.split('-')[1] + '--' + VIEW[0] + '-' + mv.split('-')[0] + '--' + VIEW[0] + '-' + mv.split('-')[1]
		print(f'{"Executing ("+str(counter)+"/"+str(len(view_id)*len(CRYPTO))+")":<20}{view_name:<70}{c.upper():<15}')
		snapshot_left = get_raw_data(DATA_PATH, VIEW[1], mv.split('-')[0], c)
		snapshot_right = get_raw_data(DATA_PATH, VIEW[1], mv.split('-')[1], c)
		candle_left = get_raw_data(DATA_PATH, VIEW[0], mv.split('-')[0], c)
		candle_right = get_raw_data(DATA_PATH, VIEW[0], mv.split('-')[1], c)
		merge_side = 'left' if len(snapshot_left) > len(snapshot_right) else 'right'
		data_left = snapshot_left.merge(snapshot_right, on='origin_time', how=merge_side, suffixes=('', '_1'))
		merge_side = 'left' if len(candle_left) > len(candle_right) else 'right'
		data_right = candle_left.merge(candle_right, on='origin_time', how=merge_side, suffixes=('', '_1'))
		data = data_left.merge(data_right, on='origin_time', how='left', suffixes=('', '_2'))
		common = [i for i in data_left.columns if i in data_right.columns]
		columns_to_drop = columns_to_drop + [i+'_2' for i in common[1:]]
		X_train, y_train, X_test, y_test = get_train_test_data(data, columns_to_drop)
		del snapshot_left, snapshot_right, candle_left, candle_right, data_left, data_right, data
		results = train_test_model(X_train, y_train, X_test, y_test)
		save_results(REPORT_FILE, results, view_name, c)
		counter += 1

## **Group 6**

Executing experiments on sixtuple views of different triples of the same type (i.e. **snapshop-snapshot-snapshot-candle-candle-candle** ).

In [32]:
# WARNING: OUT-OF-MEMORY EXPERIMENT

columns_to_drop = FEATURES_EXCLUDED[VIEW[1]] + [i+'_1' for i in FEATURES_EXCLUDED[VIEW[1]][1:]] + [i+'_2' for i in FEATURES_EXCLUDED[VIEW[1]][1:]] + FEATURES_EXCLUDED[VIEW[0]] + [i+'_1' for i in FEATURES_EXCLUDED[VIEW[0]][1:]] + [i+'_2' for i in FEATURES_EXCLUDED[VIEW[0]][1:]]
columns_to_drop = list(set(columns_to_drop))
for i, c in enumerate(CRYPTO):
	view_name = 'snapshot-BINANCE--snapshot-HOUBI--snapshot-OKX--candle-BINANCE--candle-HOUBI--candle-OKX'
	print(f'{"Executing ("+str(i+1)+"/"+str(len(CRYPTO))+")":<18}{view_name:<90}{c.upper():<10}')
	snapshot_binance = get_raw_data(DATA_PATH, VIEW[1], EXCHANGE[0], c)
	snapshot_houbi = get_raw_data(DATA_PATH, VIEW[1], EXCHANGE[1], c)
	snapshot_okx = get_raw_data(DATA_PATH, VIEW[1], EXCHANGE[2], c)
	candle_binance = get_raw_data(DATA_PATH, VIEW[0], EXCHANGE[0], c)
	candle_houbi = get_raw_data(DATA_PATH, VIEW[0], EXCHANGE[1], c)
	candle_okx = get_raw_data(DATA_PATH, VIEW[0], EXCHANGE[2], c)
	merge_side = 'left' if len(snapshot_binance) > len(snapshot_houbi) else 'right'
	data_left = snapshot_binance.merge(snapshot_houbi, on='origin_time', how=merge_side, suffixes=('', '_1'))
	data_left = data_left.merge(snapshot_okx, on='origin_time', how='left', suffixes=('', '_2'))
	merge_side = 'left' if len(candle_binance) > len(candle_houbi) else 'right'
	data_right = candle_binance.merge(candle_houbi, on='origin_time', how=merge_side, suffixes=('', '_1'))
	data_right = data_right.merge(candle_okx, on='origin_time', how='left', suffixes=('', '_2'))
	data = data_left.merge(data_right, on='origin_time', how='left', suffixes=('', '_3'))
	common = [i for i in data_left.columns if i in data_right.columns]
	columns_to_drop = columns_to_drop + [i+'_3' for i in common[1:]]
	X_train, y_train, X_test, y_test = get_train_test_data(data, columns_to_drop)
	del snapshot_binance, snapshot_houbi, snapshot_okx, candle_binance, candle_houbi, candle_okx, data_left, data_right, data
	results = train_test_model(X_train, y_train, X_test, y_test)
	save_results(REPORT_FILE, results, view_name, c)